<a href="https://colab.research.google.com/github/NewTAs/GoogLeNet/blob/main/GoogLeNet_Step3_DriveVer_Epoch10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

* 참고: https://tykimos.github.io/2017/03/08/CNN_Getting_Started/

In [ ]:
# Step1 Code
import os
import re
import glob
import cv2
import numpy as np
from sklearn.model_selection import train_test_split

import tensorflow as tf

from google.colab import drive
drive.mount('googlenet')

# New Cord
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator

np.random.seed(4) # 랜덤시드 고정시키기

Mounted at googlenet


In [ ]:
pictureWeightHeightSize = 227

train_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        '/content/googlenet/MyDrive/GoogLeNet_Data/TrainData_Step3/',
        target_size=(pictureWeightHeightSize, pictureWeightHeightSize),
        batch_size=4,
        class_mode='categorical')

test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
        '/content/googlenet/MyDrive/GoogLeNet_Data/TestData_Step3/',
        target_size=(pictureWeightHeightSize, pictureWeightHeightSize),    
        batch_size=4,
        class_mode='categorical')

Found 60 images belonging to 4 classes.
Found 12 images belonging to 4 classes.


In [ ]:
input_layer = tf.keras.layers.Input(shape=(227, 227, 3))

In [ ]:
stem_layer = tf.keras.layers.Conv2D(32, (3, 3), strides=(1, 1), padding='SAME', activation='relu')(input_layer)
# Max Pooling Here on Original Inception (3x3, stride=2)
stem_layer = tf.keras.layers.Conv2D(64, (3, 3), strides=(1, 1), padding='SAME', activation='relu')(stem_layer)
# Max Pooling Here on Original Inception (3x3, stride=2)

In [ ]:
inception_block3a_1x1 = tf.keras.layers.Conv2D(32, (1, 1), strides=(1, 1), padding='SAME', activation='relu')(stem_layer)

inception_block3a_3x3 = tf.keras.layers.Conv2D(48, (1, 1), padding='SAME', activation='relu')(stem_layer)
inception_block3a_3x3 = tf.keras.layers.Conv2D(64, (3, 3), padding='SAME', activation='relu')(inception_block3a_3x3)

inception_block3a_5x5 = tf.keras.layers.Conv2D(8, (1, 1), padding='SAME', activation='relu')(stem_layer)
inception_block3a_5x5 = tf.keras.layers.Conv2D(16, (5, 5), padding='SAME', activation='relu')(inception_block3a_5x5)

inception_block3a_pool = tf.keras.layers.MaxPool2D((3, 3), strides=(1, 1), padding='SAME')(stem_layer)
inception_block3a_pool = tf.keras.layers.Conv2D(16, (1, 1), padding='SAME', activation='relu')(inception_block3a_pool)

inception_block3a = tf.keras.layers.Concatenate()([inception_block3a_1x1, inception_block3a_3x3,
                                                   inception_block3a_5x5, inception_block3a_pool]) # 32x32x128

In [ ]:
# inception_block3b Here on Original Inception
max_pool = tf.keras.layers.MaxPool2D((3, 3), strides=(2, 2), padding='SAME')(inception_block3a) # 16x16x128

In [ ]:
inception_block4a_1x1 = tf.keras.layers.Conv2D(64, (1, 1), padding='SAME', activation='relu')(max_pool)

inception_block4a_3x3 = tf.keras.layers.Conv2D(64, (1, 1), padding='SAME', activation='relu')(max_pool)
inception_block4a_3x3 = tf.keras.layers.Conv2D(96, (3, 3), padding='SAME', activation='relu')(inception_block4a_3x3)

inception_block4a_5x5 = tf.keras.layers.Conv2D(16, (1, 1), padding='SAME', activation='relu')(max_pool)
inception_block4a_5x5 = tf.keras.layers.Conv2D(48, (5, 5), padding='SAME', activation='relu')(inception_block4a_5x5)

inception_block4a_pool = tf.keras.layers.MaxPool2D((3, 3), strides=(1, 1), padding='SAME')(max_pool)
inception_block4a_pool = tf.keras.layers.Conv2D(32, (1, 1), padding='SAME', activation='relu')(inception_block4a_pool)

inception_block4a = tf.keras.layers.Concatenate()([inception_block4a_1x1, inception_block4a_3x3,
                                                   inception_block4a_5x5, inception_block4a_pool]) # 16x16x240

# inception_block4b Here on Original Inception
# inception_block4c Here on Original Inception
# inception_block4d Here on Original Inception
# inception_block4e Here on Original Inception

In [ ]:
max_pool = tf.keras.layers.MaxPool2D((3, 3), strides=(2, 2), padding='SAME')(inception_block4a) # 8x8x240

In [ ]:
inception_block5a_1x1 = tf.keras.layers.Conv2D(128, (1, 1), padding='SAME', activation='relu')(max_pool)

inception_block5a_3x3 = tf.keras.layers.Conv2D(80, (1, 1), padding='SAME', activation='relu')(max_pool)
inception_block5a_3x3 = tf.keras.layers.Conv2D(120, (3, 3), padding='SAME', activation='relu')(inception_block5a_3x3)

inception_block5a_5x5 = tf.keras.layers.Conv2D(16, (1, 1), padding='SAME', activation='relu')(max_pool)
inception_block5a_5x5 = tf.keras.layers.Conv2D(64, (5, 5), padding='SAME', activation='relu')(inception_block5a_5x5)

inception_block5a_pool = tf.keras.layers.MaxPool2D((3, 3), strides=(1, 1), padding='SAME')(max_pool)
inception_block5a_pool = tf.keras.layers.Conv2D(64, (1, 1), padding='SAME', activation='relu')(inception_block5a_pool)

inception_block5a = tf.keras.layers.Concatenate()([inception_block5a_1x1, inception_block5a_3x3,
                                                   inception_block5a_5x5, inception_block5a_pool]) # 8x8x416
# inception_block5b Here on Original Inception

In [ ]:
avg_pool = tf.keras.layers.GlobalAveragePooling2D()(inception_block5a) # 416
dropout = tf.keras.layers.Dropout(0.4)(avg_pool)
out_layer = tf.keras.layers.Dense(4, activation='softmax')(dropout)

In [ ]:
model = tf.keras.models.Model(input_layer, out_layer)

#model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

#model.fit(x_train, y_train, batch_size=128, epochs=10, validation_data=(x_test, y_test))

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 227, 227, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 227, 227, 32) 896         input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 227, 227, 64) 18496       conv2d[0][0]                     
__________________________________________________________________________________________________
conv2d_3 (Conv2D)               (None, 227, 227, 48) 3120        conv2d_1[0][0]                   
______________________________________________________________________________________________

In [ ]:
model.fit(train_generator, epochs=10, validation_data=test_generator)

Epoch 1/10
15/15 [==============================] - 67s 4s/step - loss: 0.3978 - accuracy: 0.7833 - val_loss: 0.6481 - val_accuracy: 0.7500
Epoch 2/10
15/15 [==============================] - 67s 4s/step - loss: 0.3862 - accuracy: 0.8167 - val_loss: 0.5560 - val_accuracy: 0.7500
Epoch 3/10
15/15 [==============================] - 67s 4s/step - loss: 0.4204 - accuracy: 0.7500 - val_loss: 0.5486 - val_accuracy: 0.7500
Epoch 4/10
15/15 [==============================] - 67s 4s/step - loss: 0.3810 - accuracy: 0.8167 - val_loss: 1.3605 - val_accuracy: 0.5000
Epoch 5/10
15/15 [==============================] - 66s 4s/step - loss: 0.2805 - accuracy: 0.8500 - val_loss: 1.5025 - val_accuracy: 0.5000
Epoch 6/10
15/15 [==============================] - 66s 4s/step - loss: 0.2125 - accuracy: 0.9333 - val_loss: 0.8830 - val_accuracy: 0.7500
Epoch 7/10
15/15 [==============================] - 66s 4s/step - loss: 0.1237 - accuracy: 0.9667 - val_loss: 0.6696 - val_accuracy: 0.7500
Epoch 8/10
15/15 [==

In [ ]:
print("-- Evaluate --")
scores = model.evaluate(test_generator, steps=3)
print("%s: %.2f%%" %(model.metrics_names[1], scores[1]*100))

-- Evaluate --
3/3 [==============================] - 3s 877ms/step - loss: 1.3753 - accuracy: 0.7500
accuracy: 75.00%


In [ ]:
print("-- Predict --")
output = model.predict(test_generator, steps=3)
np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)})
print(test_generator.class_indices)
print(output)

-- Predict --
{'1': 0, '2': 1, '3': 2, '4': 3}
{'1': 0, '2': 1, '3': 2, '4': 3}
[[0.015 0.962 0.023 0.000]
 [0.015 0.960 0.026 0.000]
 [0.000 0.017 0.983 0.000]
 [0.004 0.260 0.736 0.000]
 [0.000 0.017 0.983 0.000]
 [0.004 0.244 0.752 0.000]
 [0.000 0.000 0.000 1.000]
 [0.000 0.000 0.000 1.000]
 [0.000 0.000 0.000 1.000]
 [0.005 0.277 0.718 0.000]
 [0.000 0.017 0.983 0.000]
 [0.014 0.957 0.029 0.000]]
